In [0]:
import numpy as np


In [125]:
f = open('twink.txt', 'rb')
lines=[]
for line in f: # 데이터를 한 줄씩 읽는다.
    line=line.strip() # strip()을 통해 \r, \n을 제거한다.
    line=line.lower() # 소문자화.
    line=line.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(line) > 0:
        lines.append(line)
f.close()


lines[:5]

['twinkle, twinkle, little star,',
 'how i wonder what you are.',
 'up above the world so high,',
 'like a diamond in the sky.',
 'twinkle, twinkle, little star,']

In [126]:
alice_story = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(alice_story))

문자열의 길이 또는 총 글자의 개수: 710


In [127]:
print(alice_story[:200])

twinkle, twinkle, little star, how i wonder what you are. up above the world so high, like a diamond in the sky. twinkle, twinkle, little star, how i wonder what you are! when the blazing sun is gone,


In [128]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf


In [0]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed


In [0]:
from tensorflow.keras.utils import to_categorical

In [131]:
tokens = alice_story.split() # '\n 제거'
alice_story = ' '.join(tokens)
print(alice_story)


twinkle, twinkle, little star, how i wonder what you are. up above the world so high, like a diamond in the sky. twinkle, twinkle, little star, how i wonder what you are! when the blazing sun is gone, when he nothing shines upon, then you show your little light, twinkle, twinkle, all the night. twinkle, twinkle, little star, how i wonder what you are! then the traveler in the dark thanks you for your tiny spark; how could he see where to go? if you did not twinkle so. twinkle, twinkle, little star, how i wonder what you are! in the dark blue sky you keep, while you through my window peep, and you never shut your eye, till the sun is in the sky, twinkle, twinkle, little star, how i wonder what you are!


In [132]:
char_vocab = sorted(list(set(alice_story))) # 중복을 제거한 글자 집합 생성
print(char_vocab)

[' ', '!', ',', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


In [133]:
vocab_size=len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 29


In [134]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, ',': 2, '.': 3, ';': 4, '?': 5, 'a': 6, 'b': 7, 'c': 8, 'd': 9, 'e': 10, 'f': 11, 'g': 12, 'h': 13, 'i': 14, 'k': 15, 'l': 16, 'm': 17, 'n': 18, 'o': 19, 'p': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'y': 27, 'z': 28}


In [135]:
length = 11
sequences = []
for i in range(length, len(alice_story)):
    seq = alice_story[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 699


In [136]:
sequences[:10]

['twinkle, tw',
 'winkle, twi',
 'inkle, twin',
 'nkle, twink',
 'kle, twinkl',
 'le, twinkle',
 'e, twinkle,',
 ', twinkle, ',
 ' twinkle, l',
 'twinkle, li']

In [0]:
X = []
for line in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    temp_X = [char_to_index[char] for char in line] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    X.append(temp_X)

In [138]:
for line in X[:5]:
    print(line)

[23, 26, 14, 18, 15, 16, 10, 2, 0, 23, 26]
[26, 14, 18, 15, 16, 10, 2, 0, 23, 26, 14]
[14, 18, 15, 16, 10, 2, 0, 23, 26, 14, 18]
[18, 15, 16, 10, 2, 0, 23, 26, 14, 18, 15]
[15, 16, 10, 2, 0, 23, 26, 14, 18, 15, 16]


In [0]:
sequences = np.array(X)
X = sequences[:,:-1]
y = sequences[:,-1] # 맨 마지막 위치의 글자를 분리

In [0]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) # y에 대한 원-핫 인코딩

In [141]:
print(X.shape)

(699, 10, 29)


In [142]:
alice_story[:200]

'twinkle, twinkle, little star, how i wonder what you are. up above the world so high, like a diamond in the sky. twinkle, twinkle, little star, how i wonder what you are! when the blazing sun is gone,'

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
model = Sequential()
model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2]))) # X.shape[1]은 25, X.shape[2]는 33
model.add(Dense(vocab_size, activation='softmax'))

In [145]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=150, verbose=2)

Train on 699 samples
Epoch 1/150
699/699 - 2s - loss: 3.2933 - accuracy: 0.1631
Epoch 2/150
699/699 - 0s - loss: 2.9669 - accuracy: 0.1903
Epoch 3/150
699/699 - 0s - loss: 2.8243 - accuracy: 0.1903
Epoch 4/150
699/699 - 0s - loss: 2.7944 - accuracy: 0.1903
Epoch 5/150
699/699 - 0s - loss: 2.7538 - accuracy: 0.1903
Epoch 6/150
699/699 - 0s - loss: 2.7148 - accuracy: 0.1903
Epoch 7/150
699/699 - 0s - loss: 2.6626 - accuracy: 0.2132
Epoch 8/150
699/699 - 0s - loss: 2.6147 - accuracy: 0.2332
Epoch 9/150
699/699 - 0s - loss: 2.5607 - accuracy: 0.2461
Epoch 10/150
699/699 - 0s - loss: 2.4927 - accuracy: 0.2718
Epoch 11/150
699/699 - 0s - loss: 2.4193 - accuracy: 0.2947
Epoch 12/150
699/699 - 0s - loss: 2.3487 - accuracy: 0.3190
Epoch 13/150
699/699 - 0s - loss: 2.2781 - accuracy: 0.3247
Epoch 14/150
699/699 - 0s - loss: 2.1968 - accuracy: 0.3548
Epoch 15/150
699/699 - 0s - loss: 2.1288 - accuracy: 0.3777
Epoch 16/150
699/699 - 0s - loss: 2.0811 - accuracy: 0.4177
Epoch 17/150
699/699 - 0s - 

In [0]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
# 모델, 인덱스 정보, 문장 길이, 초기 시퀀스, 반복 횟수
    init_text = seed_text # 문장 생성에 사용할 초기 시퀀스
    sentence = ''

    for _ in range(n): # n번 반복
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        for char, index in char_to_index.items(): # 만약 예측한 글자와 인덱스와 동일한 글자가 있다면
            if index == result: # 해당 글자가 예측 글자이므로 break
                break
        seed_text=seed_text + char # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        sentence=sentence + char # 예측 글자를 문장에 저장
        # for문이므로 이 작업을 다시 반복

    sentence = init_text + sentence
    return sentence

In [150]:
print(sentence_generation(model, char_to_index, 10, 'the traveler in', 60))

the traveler in the dark thanks you for your tiny spark; how could he see w
